## Amazon Review - Zero Shot - Error EDA

This notebook is for the error analysis and eda.

### Load Packages

In [2]:
import pandas as pd
import random
import seaborn as sns
import numpy as np

### Load in Data

In [3]:
file_name = '../../test_predictions.pkl'
test_results = pd.read_pickle(file_name)

file_name = '../../imdb_test_predictions.pkl'
imdb_results = pd.read_pickle(file_name)

file_name = '../../twitter_sent140_test_predictions.pkl'
twitter_results = pd.read_pickle(file_name)

file_name = '../../yelp_test_predictions.pkl'
yelp_results = pd.read_pickle(file_name)

file_name = '../../yelp_zhang_test_predictions.pkl'
yelp_zhang_results = pd.read_pickle(file_name)

file_name = '../../amazon_zhang_test_predictions.pkl'
amazon_zhang_results = pd.read_pickle(file_name)

file_name = '../../sst_test_predictions.pkl'
sst_results = pd.read_pickle(file_name)

In [4]:
def appendPreds(data):
    cols = ['USE_Soft', 'CNN_TE', 'CNN_TE_USE', 'CNN_Glove', 'CNN_Glove_USE',
        'CNN_w2V', 'CNN_w2V_USE', 'Individual_Sent', 'Average_Sent']
    preds = ['USE_Soft_pred', 'CNN_TE_pred', 'CNN_TE_USE_pred', 'CNN_Glove_pred', 'CNN_Glove_USE_pred',
        'CNN_w2V_pred', 'CNN_w2V_USE_pred', 'Individual_Sent_pred', 'Average_Sent_pred']
    
    for col, pred in zip(cols,preds):
        data[pred] = np.where(data[col]>=0.5,1,0)
        
    return data

test_results = appendPreds(test_results)

imdb_results = appendPreds(imdb_results)

twitter_results = appendPreds(twitter_results)

yelp_results = appendPreds(yelp_results)

yelp_zhang_results = appendPreds(yelp_zhang_results)

amazon_zhang_results = appendPreds(amazon_zhang_results)

sst_results = appendPreds(sst_results)

In [5]:
def checkDifs(data, cutoff = 2, Indv = True, Avg = True):
    
    n = data.shape[0]
    newDF = data
    preds = ['USE_Soft_pred', 'CNN_TE_pred', 'CNN_TE_USE_pred', 'CNN_Glove_pred', 'CNN_Glove_USE_pred',
        'CNN_w2V_pred', 'CNN_w2V_USE_pred']
    
    indiv = ['Individual_Sent_pred']
    avg = ['Average_Sent_pred']
    
    USE = [True, False, True, False, True, False, True, True, True]
    
    newDF['Preds_Agree3'] = np.where((newDF['CNN_TE_pred']==newDF['CNN_Glove_pred'])
                                    & (newDF['CNN_Glove_pred']==newDF['CNN_w2V_pred']),1,0)
    newDF['Preds_Agree2'] = np.where((newDF['CNN_TE_pred']==newDF['CNN_Glove_pred'])
                                    | (newDF['CNN_Glove_pred']==newDF['CNN_w2V_pred'])
                                    | (newDF['CNN_TE_pred']==newDF['CNN_w2V_pred']),1,0)
    
    newDF['USE_Preds_Agree3'] = np.where((newDF['CNN_TE_USE_pred']==newDF['CNN_Glove_USE_pred'])
                                    & (newDF['CNN_Glove_USE_pred']==newDF['CNN_w2V_USE_pred']),1,0)
    newDF['USE_Preds_Agree2'] = np.where((newDF['CNN_TE_USE_pred']==newDF['CNN_Glove_USE_pred'])
                                    | (newDF['CNN_Glove_USE_pred']==newDF['CNN_w2V_USE_pred'])
                                    | (newDF['CNN_TE_USE_pred']==newDF['CNN_w2V_USE_pred']),1,0)
    
    newDF['dis_3'] = np.where((newDF['Preds_Agree3']==newDF['USE_Preds_Agree3'])
                                    & (((newDF['CNN_Glove_USE_pred']!=newDF['sentiment']) & 
                                      (newDF['CNN_Glove_pred']==newDF['sentiment'])) |
                                    ((newDF['CNN_Glove_USE_pred']==newDF['sentiment']) & 
                                      (newDF['CNN_Glove_pred']!=newDF['sentiment']))),1,0)
    return n, newDF

In [6]:
n, test_results = checkDifs(test_results)
USE_imp_test = test_results.loc[(test_results['USE_Preds_Agree3'] == 1) & (test_results['Preds_Agree3'] == 1)]

In [7]:
USE_imp_test = USE_imp_test.drop(['Average_Sent_pred','Individual_Sent_pred','Preds_Agree2','USE_Preds_Agree2',], axis = 1)

In [8]:
count_use = USE_imp_test.loc[((USE_imp_test['dis_3'] == 1) & (USE_imp_test['CNN_TE_USE_pred'] == USE_imp_test['sentiment']))]
count_norm = USE_imp_test.loc[((USE_imp_test['dis_3'] == 1) & (USE_imp_test['CNN_TE_pred'] == USE_imp_test['sentiment']))]

In [9]:
n_use = count_use.shape[0]
n_norm = count_norm.shape[0]


In [10]:
n_use

148

In [11]:
n_norm

48

## IMDB Results

In [17]:
n, _imdb_results = checkDifs(imdb_results)
USE_imp_imdb_results = _imdb_results.loc[(_imdb_results['USE_Preds_Agree3'] == 1) & (_imdb_results['Preds_Agree3'] == 1)]

USE_imp_imdb_results = USE_imp_imdb_results.drop(['Average_Sent_pred','Individual_Sent_pred','Preds_Agree2','USE_Preds_Agree2',], axis = 1)

count_use_imdb = USE_imp_imdb_results.loc[((USE_imp_imdb_results['dis_3'] == 1) & (USE_imp_imdb_results['CNN_TE_USE_pred'] == USE_imp_imdb_results['sentiment']))]
count_norm_imdb = USE_imp_imdb_results.loc[((USE_imp_imdb_results['dis_3'] == 1) & (USE_imp_imdb_results['CNN_TE_pred'] == USE_imp_imdb_results['sentiment']))]

n_use_imdb = count_use_imdb.shape[0]
n_norm_imdb = count_norm_imdb.shape[0]

print(n_use_imdb)
print(n_norm_imdb)

33
10


### Check When USE was Correct

In [23]:
check_lens = [len(words) for words in count_use_imdb['sentenceWords']]

med_use = np.median(check_lens)
print(med_use)
print(list(count_use_imdb['sentenceWords'][:200].iloc[[np.argmax(check_lens)]]))
print('\n_________________\n')
print(list(count_use_imdb['reviewText'].iloc[[np.argmax(check_lens)]]))
print('\nGlove prediction = {}'.format(list(count_use_imdb['CNN_Glove'].iloc[[np.argmax(check_lens)]])))
print('Glove + USE prediction = {}'.format(list(count_use_imdb['CNN_Glove_USE'].iloc[[np.argmax(check_lens)]])))

200.0
[['when', 'one', 'stops', 'to', 'recollect', 'upon', 'the', 'frequent', 'on', 'screen', 'teaming', 'of', 'errol', 'flynn', 'and', 'olivia', 'dehavilland', 'they', 'died', 'with', 'their', 'boots', 'on', 'is', 'most', 'likely', 'the', 'film', 'remembered', 'best', 'it', 'is', 'the', 'sweeping', 'saga', 'of', 'general', 'custer', 'flynn', 'told', 'from', 'the', 'time', 'he', 'enters', 'west', 'point', 'military', 'academy', 'and', 'falls', 'for', 'the', 'luscious', 'elizabeth', 'bacon', 'dehavilland', 'through', 'his', 'tenure', 'during', 'the', 'american', 'civil', 'war', 'and', 'finally', 'with', 'his', 'death', 'at', 'little', 'big', 'horn', 'director', 'raoul', 'walsh', 'mounts', 'his', 'historical', 'epic', 'on', 'the', 'laurels', 'of', 'highly', 'questionable', 'recanting', 'of', 'historical', 'texts', 'rewritten', 'by', 'screen', 'writers', 'wally', 'kline', 'and', 'aeneas', 'mackenzie', 'until', 'truth', 'and', 'fiction', 'are', 'warped', 'all', 'out', 'of', 'proportion', '

In [24]:
print(list(count_use_imdb['sentenceWords'][:200].iloc[[np.argmin(check_lens)]]))
print('\n_________________\n')
print(list(count_use_imdb['reviewText'].iloc[[np.argmin(check_lens)]]))
print('\nGlove prediction = {}'.format(list(count_use_imdb['CNN_Glove'].iloc[[np.argmin(check_lens)]])))
print('Glove + USE prediction = {}'.format(list(count_use_imdb['CNN_Glove_USE'].iloc[[np.argmin(check_lens)]])))

[['farscape', 'totally', 'rules', 'in', 'my', 'opinion', 'it', 'is', 'very', 'close', 'to', 'babylon', 'five', 'although', 'there', 'are', 'only', 'main', 'characters', 'in', 'the', 'series', 'spaceship', 'included', 'the', 'humour', 'is', 'excellent', 'and', 'the', 'writers', 'manage', 'to', 'keep', 'the', 'show', 'interesting', 'though', 'pretty', 'much', 'everything', 'happens', 'on', 'the', 'ship', 'br', 'br', 'what', 'i', 'really', 'love', 'and', 'appreciate', 'in', 'farscpae', 'is', 'that', 'they', 'do', 'not', 'use', 'cgi', 'for', 'the', 'alien', 'characters', 'or', 'if', 'they', 'do', 'it', 'is', 'unnoticeable', 'they', 'use', 'those', 'lovable', 'crappy', 'rubber', 'animatronic', 'puppets', 'very', 'similar', 'to', 'the', 'ones', 'used', 'in', 'the', 'original', 'star', 'wars', 'br', 'br', 'farscape', 'is', 'something', 'you', 'definitely', 'want', 'to', 'look', 'into', 'if', 'you', 'already', 'have', 'not']]

_________________

["Farscape totally rules! In my opinion it's ver

In [73]:
count_use_imdb['avg_base'] = (count_use_imdb['CNN_Glove'] + count_use_imdb['CNN_TE'] + count_use_imdb['CNN_w2V'])/3
count_use_imdb['avg_USE'] = (count_use_imdb['CNN_Glove_USE'] + count_use_imdb['CNN_TE_USE'] + count_use_imdb['CNN_w2V_USE'])/3

count_use_imdb['diffscore'] = abs(count_use_imdb['avg_base'] - count_use_imdb['avg_USE'])

/home/lance_miles/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/lance_miles/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/lance_miles/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

In [74]:
count_use_imdb_diff = [prob for prob in count_use_imdb['diffscore']]

print(list(count_use_imdb['sentenceWords'][:200].iloc[[np.argmax(count_use_imdb_diff)]]))
print('\n_________________\n')
print(list(count_use_imdb['reviewText'].iloc[[np.argmax(count_use_imdb_diff)]]))
print('\n_________________\n\nSentiment = {}'.format(list(count_use_imdb['sentiment'].iloc[[np.argmax(count_use_imdb_diff)]])))
print('\n_________________\n\nAverage Baseline Prediction = {}'.format(list(count_use_imdb['avg_base'].iloc[[np.argmax(count_use_imdb_diff)]])))
print('Average Baseline + USE Prediction = {}'.format(list(count_use_imdb['avg_USE'].iloc[[np.argmax(count_use_imdb_diff)]])))

[['when', 'one', 'stops', 'to', 'recollect', 'upon', 'the', 'frequent', 'on', 'screen', 'teaming', 'of', 'errol', 'flynn', 'and', 'olivia', 'dehavilland', 'they', 'died', 'with', 'their', 'boots', 'on', 'is', 'most', 'likely', 'the', 'film', 'remembered', 'best', 'it', 'is', 'the', 'sweeping', 'saga', 'of', 'general', 'custer', 'flynn', 'told', 'from', 'the', 'time', 'he', 'enters', 'west', 'point', 'military', 'academy', 'and', 'falls', 'for', 'the', 'luscious', 'elizabeth', 'bacon', 'dehavilland', 'through', 'his', 'tenure', 'during', 'the', 'american', 'civil', 'war', 'and', 'finally', 'with', 'his', 'death', 'at', 'little', 'big', 'horn', 'director', 'raoul', 'walsh', 'mounts', 'his', 'historical', 'epic', 'on', 'the', 'laurels', 'of', 'highly', 'questionable', 'recanting', 'of', 'historical', 'texts', 'rewritten', 'by', 'screen', 'writers', 'wally', 'kline', 'and', 'aeneas', 'mackenzie', 'until', 'truth', 'and', 'fiction', 'are', 'warped', 'all', 'out', 'of', 'proportion', 'hence'

In [75]:
count_use_imdb_diff = [prob for prob in count_use_imdb['diffscore']]

print(list(count_use_imdb['sentenceWords'][:200].iloc[[np.argmin(count_use_imdb_diff)]]))
print('\n_________________\n')
print(list(count_use_imdb['reviewText'].iloc[[np.argmin(count_use_imdb_diff)]]))
print('\n_________________\n\nSentiment = {}'.format(list(count_use_imdb['sentiment'].iloc[[np.argmin(count_use_imdb_diff)]])))
print('\n_________________\n\nAverage Baseline Prediction = {}'.format(list(count_use_imdb['avg_base'].iloc[[np.argmin(count_use_imdb_diff)]])))
print('Average Baseline + USE Prediction = {}'.format(list(count_use_imdb['avg_USE'].iloc[[np.argmin(count_use_imdb_diff)]])))

[['this', 'is', 'a', 'hollywood', 'film', 'the', 'credentials', 'of', 'those', 'involved', 'are', 'not', 'revolutionary', 'this', 'film', 'will', 'not', 'change', 'the', 'world', 'however', 'it', 'is', 'well', 'made', 'well', 'plotted', 'and', 'well', 'acted', 'which', 'counts', 'for', 'a', 'lot', 'br', 'br', 'it', 'is', 'the', 'cat', 'and', 'mouse', 'story', 'of', 'a', 'disgruntled', 'former', 'cia', 'assassin', 'who', 'feels', 'betrayed', 'chasing', 'a', 'secret', 'service', 'agent', 'who', 'was', 'protecting', 'john', 'f', 'kennedy', 'when', 'kennedy', 'was', 'killed', 'in', 'dallas', 'the', 'agent', 'has', 'been', 'understandably', 'troubled', 'by', 'this', 'failure', 'during', 'the', 'subsequent', 'years', 'of', 'his', 'career', 'and', 'when', 'the', 'movie', 'picks', 'up', 'in', 'or', 'certainly', 'in', 'contemporary', 'america', 'this', 'would', 'be', 'assassin', 'mitch', 'leary', 'john', 'malkovich', 'in', 'a', 'tour', 'de', 'force', 'of', 'creepiness', 'in', 'which', 'he', 'gi

### Check when Baselines were Correct

In [33]:
check_lens_imdb_norm = [len(words) for words in count_norm_imdb['sentenceWords']]

med_use = np.median(check_lens_imdb_norm)
print('Median sentence length = {}\n'.format(med_use))

print(list(count_norm_imdb['sentenceWords'][:200].iloc[[np.argmax(check_lens_imdb_norm)]]))
print('\n_________________\n')
print(list(count_norm_imdb['reviewText'].iloc[[np.argmax(check_lens_imdb_norm)]]))
print('\n_________________\n\nGlove prediction = {}'.format(list(count_norm_imdb['CNN_Glove'].iloc[[np.argmax(check_lens_imdb_norm)]])))
print('Glove + USE prediction = {}'.format(list(count_norm_imdb['CNN_Glove_USE'].iloc[[np.argmax(check_lens_imdb_norm)]])))

Median sentence length = 200.0

[['st', 'watched', 'out', 'of', 'dir', 'stefan', 'rujowitzky', 'ok', 'thriller', 'but', 'a', 'little', 'too', 'predictable', 'this', 'story', 'is', 'based', 'in', 'germany', 'which', 'is', 'also', 'where', 'the', 'movie', 'is', 'made', 'it', 'is', 'about', 'a', 'young', 'medical', 'student', 'who', 'gets', 'a', 'shot', 'to', 'go', 'to', 'a', 'premiere', 'school', 'in', 'heidelberg', 'and', 'arrives', 'seeing', 'some', 'strange', 'things', 'occurring', 'someone', 'she', 'met', 'on', 'the', 'train', 'there', 'and', 'saved', 'shows', 'up', 'on', 'the', 'school', 'is', 'experimentation', 'table', 'and', 'she', 'is', 'suspecting', 'foul', 'play', 'right', 'away', 'she', 'does', 'some', 'investigation', 'and', 'the', 'disappearance', 'of', 'her', 'friend', 'leads', 'her', 'to', 'a', 'secret', 'society', 'called', 'aaa', 'and', 'no', 'it', 'is', 'not', 'alcoholics', 'anonymous', 'that', 'has', 'something', 'to', 'do', 'with', 'the', 'anti', 'hippocratic', 'oath

In [34]:
print(list(count_norm_imdb['sentenceWords'][:200].iloc[[np.argmin(check_lens_imdb_norm)]]))
print('\n_________________\n')
print(list(count_norm_imdb['reviewText'].iloc[[np.argmin(check_lens_imdb_norm)]]))
print('\n_________________\n\nGlove prediction = {}'.format(list(count_norm_imdb['CNN_Glove'].iloc[[np.argmin(check_lens_imdb_norm)]])))
print('Glove + USE prediction = {}'.format(list(count_norm_imdb['CNN_Glove_USE'].iloc[[np.argmin(check_lens_imdb_norm)]])))

[['i', 'was', 'surprised', 'to', 'read', 'the', 'comments', 'of', 'the', 'person', 'who', 'so', 'disliked', 'this', 'film', 'it', 'really', 'is', 'quite', 'funny', 'there', 'are', 'definitely', 'a', 'few', 'laugh', 'out', 'lines', 'that', 'my', 'boyfriend', 'and', 'i', 'quote', 'to', 'each', 'other', 'some', 'of', 'the', 'situations', 'might', 'be', 'unsettling', 'bisexuality', 'drugs', 'a', 'particularly', 'strange', 'child', 'is', 'view', 'of', 'sexuality', 'but', 'believable', 'at', 'the', 'same', 'time', 'it', 'is', 'about', 'communication', 'and', 'miscommunication', 'between', 'men', 'and', 'women']]

_________________

["I was surprised to read the comments of the person who so disliked this film. It really is quite funny. There are definitely a few laugh out lines that my boyfriend and I quote to each other. Some of the situations might be unsettling (bisexuality, drugs, a particularly strange child's view of sexuality) but believable at the same time. It's about communication 

In [39]:
count_norm_imdb['avg_base'] = (count_norm_imdb['CNN_Glove'] + count_norm_imdb['CNN_TE'] + count_norm_imdb['CNN_w2V'])/3
count_norm_imdb['avg_USE'] = (count_norm_imdb['CNN_Glove_USE'] + count_norm_imdb['CNN_TE_USE'] + count_norm_imdb['CNN_w2V_USE'])/3

/home/lance_miles/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/lance_miles/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [47]:
count_norm_imdb['diffscore'] = abs(count_norm_imdb['avg_base'] - count_norm_imdb['avg_USE'])

/home/lance_miles/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [54]:
count_norm_imdb_diff = [prob for prob in count_norm_imdb['diffscore']]

print(list(count_norm_imdb['sentenceWords'][:200].iloc[[np.argmax(count_norm_imdb_diff)]]))
print('\n_________________\n')
print(list(count_norm_imdb['reviewText'].iloc[[np.argmax(count_norm_imdb_diff)]]))
print('\n_________________\n\nSentiment = {}'.format(list(count_norm_imdb['sentiment'].iloc[[np.argmax(count_norm_imdb_diff)]])))
print('\n_________________\n\nAverage Baseline Prediction = {}'.format(list(count_norm_imdb['avg_base'].iloc[[np.argmax(count_norm_imdb_diff)]])))
print('Average Baseline + USE Prediction = {}'.format(list(count_norm_imdb['avg_USE'].iloc[[np.argmax(count_norm_imdb_diff)]])))

[['st', 'watched', 'out', 'of', 'dir', 'stefan', 'rujowitzky', 'ok', 'thriller', 'but', 'a', 'little', 'too', 'predictable', 'this', 'story', 'is', 'based', 'in', 'germany', 'which', 'is', 'also', 'where', 'the', 'movie', 'is', 'made', 'it', 'is', 'about', 'a', 'young', 'medical', 'student', 'who', 'gets', 'a', 'shot', 'to', 'go', 'to', 'a', 'premiere', 'school', 'in', 'heidelberg', 'and', 'arrives', 'seeing', 'some', 'strange', 'things', 'occurring', 'someone', 'she', 'met', 'on', 'the', 'train', 'there', 'and', 'saved', 'shows', 'up', 'on', 'the', 'school', 'is', 'experimentation', 'table', 'and', 'she', 'is', 'suspecting', 'foul', 'play', 'right', 'away', 'she', 'does', 'some', 'investigation', 'and', 'the', 'disappearance', 'of', 'her', 'friend', 'leads', 'her', 'to', 'a', 'secret', 'society', 'called', 'aaa', 'and', 'no', 'it', 'is', 'not', 'alcoholics', 'anonymous', 'that', 'has', 'something', 'to', 'do', 'with', 'the', 'anti', 'hippocratic', 'oath', 'and', 'is', 'used', 'to', 'p

In [55]:
count_norm_imdb_diff = [prob for prob in count_norm_imdb['diffscore']]

print(list(count_norm_imdb['sentenceWords'][:200].iloc[[np.argmin(count_norm_imdb_diff)]]))
print('\n_________________\n')
print(list(count_norm_imdb['reviewText'].iloc[[np.argmin(count_norm_imdb_diff)]]))
print('\n_________________\n\nSentiment = {}'.format(list(count_norm_imdb['sentiment'].iloc[[np.argmin(count_norm_imdb_diff)]])))
print('\n_________________\n\nAverage Baseline Prediction = {}'.format(list(count_norm_imdb['avg_base'].iloc[[np.argmin(count_norm_imdb_diff)]])))
print('Average Baseline + USE Prediction = {}'.format(list(count_norm_imdb['avg_USE'].iloc[[np.argmin(count_norm_imdb_diff)]])))

[['i', 'was', 'surprised', 'to', 'read', 'the', 'comments', 'of', 'the', 'person', 'who', 'so', 'disliked', 'this', 'film', 'it', 'really', 'is', 'quite', 'funny', 'there', 'are', 'definitely', 'a', 'few', 'laugh', 'out', 'lines', 'that', 'my', 'boyfriend', 'and', 'i', 'quote', 'to', 'each', 'other', 'some', 'of', 'the', 'situations', 'might', 'be', 'unsettling', 'bisexuality', 'drugs', 'a', 'particularly', 'strange', 'child', 'is', 'view', 'of', 'sexuality', 'but', 'believable', 'at', 'the', 'same', 'time', 'it', 'is', 'about', 'communication', 'and', 'miscommunication', 'between', 'men', 'and', 'women']]

_________________

["I was surprised to read the comments of the person who so disliked this film. It really is quite funny. There are definitely a few laugh out lines that my boyfriend and I quote to each other. Some of the situations might be unsettling (bisexuality, drugs, a particularly strange child's view of sexuality) but believable at the same time. It's about communication 

In [40]:
df_confusion_CNN_G = createConfusion(imdb_results,'CNN_Glove_USE_pred')
df_confusion_CNN_G

CNN_Glove_USE_pred
Accuracy: 0.86076
Precision: 0.8890518505737209
Recall: 0.8244


Predicted,0,1
Actual Sentiment,,
0,11214,1286
1,2195,10305


In [42]:
df_confusion_CNN_G = createConfusion(imdb_results,'CNN_Glove_pred')
df_confusion_CNN_G

CNN_Glove_pred
Accuracy: 0.82912
Precision: 0.9092717867091126
Recall: 0.7312


Predicted,0,1
Actual Sentiment,,
0,11588,912
1,3360,9140


## Longest Sentence Where USE + embeddings correct - Test Results

Looking at the data, below. The models that included the universal sentence encoder seems to have captured sentiment better (unanimously) than the model without the sentence encoder. This could be because of a number of reasons. For one, the inclusion of stop words likely affects how the CNN interprets this review, especially because the review clearly changes sentiment throughout. In either case, it is interesting that the inclusion of the embeddings from the USE allowed the model to tease out that this was a positive sentiment when the regular CNNs (with only word level embeddings) were unable to get this one correct.

In [12]:
check_lens = [len(words) for words in count_use['sentenceWords']]

med_use = np.median(check_lens)
print(med_use)
print(list(count_use['sentenceWords'][:200].iloc[[np.argmax(check_lens)]]))
print('\n_________________\n')
print(list(count_use['reviewText'].iloc[[np.argmax(check_lens)]]))

75.0
[['five', 'stars', 'me', 'for', 'a', 'book', 'written', 'by', 'and', 'for', 'men', 'me', 'the', 'feminist', 'yep', 'yep', 'and', 'yep', 'and', 'there', 'was', 'need', 'for', 'better', 'editing', 'still', 'five', 'whole', 'stars', 'this', 'book', 'addresses', 'what', 'i', 'fight', 'for', 'equality', 'i', 'have', 'spent', 'my', 'last', 'few', 'month', 'making', 'gifts', 'either', 'crocheting', 'or', 'knitting', 'lots', 'of', 'websites', 'and', 'ebooks', 'came', 'and', 'blurred', 'before', 'my', 'eyes', 'most', 'of', 'the', 'books', 'assume', 'you', 'are', 'a', 'woman', 'a', 'house', 'wifely', 'type', 'with', 'no', 'real', 'men', 'in', 'your', 'life', 'to', 'make', 'things', 'for', 'they', 'are', 'all', 'dedicated', 'to', 'girly', 'girls', 'with', 'squishy', 'babies', 'hey', 'been', 'there', 'done', 'that', 'with', 'four', 'full', 'grown', 'off', 'spring', 'three', 'males', 'one', 'female', 'to', 'prove', 'it', 'this', 'book', 'will', 'not', 'be', 'wiped', 'off', 'my', 'kindle', 'or'

In [13]:
check_lens = [len(words) for words in count_use['sentenceWords']]



print(list(count_use['sentenceWords'][:200].iloc[[np.argmin(check_lens)]]))
print('\n_________________\n')
print(list(count_use['reviewText'].iloc[[np.argmin(check_lens)]]))

75.0
[['as', 'expected']]

_________________

['As expected']


In [15]:
check_sents = [words for words in count_use['reviewText']]
check_sents

['I cant as I never recvd this. .  Who can help me with with this issue? Thanks for your help',
 "Not only does this film was exquisitely made for a different audience by Disney, but they were able to capture the real message implicitly stated in Carrol's book; here, we are able to succumb into the fantastic wonderlands brought upon by psychedelics, and more often than not, one is constantly reminded about the useless games and parodies we set ourselves with on a daily basis. Alice realizes that something has happened, but is not able to handle all this nonsense, which in fact are just a product of her own mind. Most importantly, I believe, is to watch this film under a preferred state of consciousness as to better appreciate the subtle messages the producers wanted to put across, and luckily, by the end of the film, you can ask yourself: Who am I?",
 'I prefer to get up close and personal and this is the knive for that!  I keep at home because it is illegal to carry but I would tell a

## Confusion Matrix


In [36]:
def createConfusion(data, model):
    
    y_actu = pd.Series(data['sentiment'], name='Actual Sentiment')
    y_pred = pd.Series(data[model], name='Predicted')
    
    conf = pd.crosstab(y_actu, y_pred)
    
    true_pos = np.diag(conf) 
    print(model)
    print('Accuracy: {}'.format(np.sum(true_pos)/data.shape[0]))
    
    precision = np.sum(true_pos[1] / np.sum(conf[1], axis=0))
    recall = np.sum(true_pos[1] / np.sum(conf.iloc[1]))
    
    print('Precision: {}'.format(precision))
    print('Recall: {}'.format(recall))

    return conf


df_confusion_CNN_G = createConfusion(test_results,'CNN_Glove_USE_pred')
df_confusion_CNN_G

CNN_Glove_USE_pred
Accuracy: 0.9481791666666667
Precision: 0.9532747300907705
Recall: 0.9425583333333334


Predicted,0,1
Actual Sentiment,,
0,114456,5544
1,6893,113107


In [37]:
df_confusion_CNN_G = createConfusion(test_results,'CNN_Glove_pred')
df_confusion_CNN_G

CNN_Glove_pred
Accuracy: 0.9338166666666666
Precision: 0.9616380533484676
Recall: 0.9036833333333333


Predicted,0,1
Actual Sentiment,,
0,115674,4326
1,11558,108442


## Product Level Accuracy On Test Data

In [76]:
test_results.columns

Index(['Product', 'asin', 'helpful', 'overall', 'reviewText', 'reviewTime',
       'reviewerID', 'reviewerName', 'sentiment', 'summary', 'unixReviewTime',
       'sentenceWords_Stops', 'sentences_indiv', 'sentenceWords', 'sentences',
       'USE_Soft', 'CNN_TE', 'CNN_TE_USE', 'CNN_Glove', 'CNN_Glove_USE',
       'CNN_w2V', 'CNN_w2V_USE', 'Individual_Sent', 'Average_Sent',
       'USE_Soft_pred', 'CNN_TE_pred', 'CNN_TE_USE_pred', 'CNN_Glove_pred',
       'CNN_Glove_USE_pred', 'CNN_w2V_pred', 'CNN_w2V_USE_pred',
       'Individual_Sent_pred', 'Average_Sent_pred', 'Preds_Agree3',
       'Preds_Agree2', 'USE_Preds_Agree3', 'USE_Preds_Agree2', 'dis_3'],
      dtype='object')

In [79]:
test_results.groupby(['Product','sentiment']).mean()

overall  unixReviewTime  \
Product                             sentiment                            
reviews_Books                       0              1.0    1.289283e+09   
                                    1              5.0    1.312893e+09   
reviews_CDs_and_Vinyl               0              1.0    1.184469e+09   
                                    1              5.0    1.209008e+09   
reviews_Cell_Phones_and_Accessories 0              1.0    1.361913e+09   
                                    1              5.0    1.366796e+09   
reviews_Clothing_Shoes_and_Jewelry  0              1.0    1.363424e+09   
                                    1              5.0    1.364221e+09   
reviews_Electronics                 0              1.0    1.333408e+09   
                                    1              5.0    1.341424e+09   
reviews_Health_and_Personal_Care    0              1.0    1.353964e+09   
                                    1              5.0    1.359021e+09   
reviews_Home_and_Kitchen            0              1.0    1.335813e+09   
                                    1              5.0    1.348899e+09   
reviews_Kindle_Store                0              1.0    1.370526e+09   
                                    1              5.0    1.374883e+09   
reviews_Movies_and_TV               0              1.0    1.269129e+09   
                                    1              5.0    1.290551e+09   
reviews_Sports_and_Outdoors         0              1.0    1.351803e+09   
                                    1              5.0    1.358907e+09   
reviews_Toys_and_Games              0              1.0    1.336254e+09   
                                    1              5.0    1.344748e+09   
reviews_Video_Games                 0              1.0    1.302598e+09   
                                    1              5.0    1.304290e+09   

                                               USE_Soft    CNN_TE  CNN_TE_USE  \
Product                             sentiment                                   
reviews_Books                       0          0.183530  0.069646    0.072668   
                                    1          0.861290  0.870147    0.886168   
reviews_CDs_and_Vinyl               0          0.176998  0.078215    0.081938   
                                    1          0.841959  0.850313    0.874002   
reviews_Cell_Phones_and_Accessories 0          0.137725  0.043581    0.037746   
                                    1          0.845152  0.910792    0.913829   
reviews_Clothing_Shoes_and_Jewelry  0          0.149859  0.045892    0.044338   
                                    1          0.882820  0.908892    0.919407   
reviews_Electronics                 0          0.147559  0.045123    0.044670   
                                    1          0.828504  0.883723    0.894531   
reviews_Health_and_Personal_Care    0          0.175850  0.059031    0.057374   
                                    1          0.815650  0.866987    0.879458   
reviews_Home_and_Kitchen            0          0.153760  0.045190    0.043898   
                                    1          0.853761  0.895136    0.905801   
reviews_Kindle_Store                0          0.145445  0.060171    0.058635   
                                    1          0.873066  0.889932    0.900397   
reviews_Movies_and_TV               0          0.145485  0.061695    0.065191   
                                    1          0.846493  0.862192    0.878895   
reviews_Sports_and_Outdoors         0          0.161356  0.047389    0.046149   
                                    1          0.857318  0.887977    0.899746   
reviews_Toys_and_Games              0          0.127355  0.042268    0.040576   
                                    1          0.889313  0.900839    0.910959   
reviews_Video_Games                 0          0.139484  0.051214    0.051772   
                                    1          0.855521  0.877481    0.890464   

               

In [94]:
product_types = ['reviews_Books','reviews_CDs_and_Vinyl','reviews_Cell_Phones_and_Accessories',
                 'reviews_Clothing_Shoes_and_Jewelry','reviews_Electronics','reviews_Health_and_Personal_Care',
                 'reviews_Home_and_Kitchen','reviews_Kindle_Store','reviews_Movies_and_TV',
                 'reviews_Sports_and_Outdoors','reviews_Toys_and_Games','reviews_Video_Games']

cols = ['USE_Soft', 'CNN_TE', 'CNN_TE_USE', 'CNN_Glove', 'CNN_Glove_USE',
        'CNN_w2V', 'CNN_w2V_USE', 'Individual_Sent', 'Average_Sent']

preds = ['USE_Soft_pred', 'CNN_TE_pred', 'CNN_TE_USE_pred', 'CNN_Glove_pred', 'CNN_Glove_USE_pred',
         'CNN_w2V_pred', 'CNN_w2V_USE_pred', 'Individual_Sent_pred', 'Average_Sent_pred']

test_df_summary = pd.DataFrame(index = product_types,
                               columns=cols)

test_df_summary

,USE_Soft,CNN_TE,CNN_TE_USE,CNN_Glove,CNN_Glove_USE,CNN_w2V,CNN_w2V_USE,Individual_Sent,Average_Sent
reviews_Books,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
reviews_CDs_and_Vinyl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
reviews_Cell_Phones_and_Accessories,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
reviews_Clothing_Shoes_and_Jewelry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
reviews_Electronics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
reviews_Health_and_Personal_Care,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
reviews_Home_and_Kitchen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
reviews_Kindle_Store,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
reviews_Movies_and_TV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
reviews_Sports_and_Outdoors,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
for prod in product_types:
    for col, pred in zip(cols, preds):
        check = test_results[pred].loc[test_results['Product'] == prod]
        sent = test_results['sentiment'].loc[test_results['Product'] == prod]

        bools = np.where(check == sent, 1,0)
#         print(bools)
        sum_acc = np.sum(bools)
        total = len(bools)
        acc = round(float(sum_acc/total)*100,3)
#         print(sum_acc)
#         print(total)
        test_df_summary.at[prod, col] = acc

        

In [105]:
test_df_summary

,USE_Soft,CNN_TE,CNN_TE_USE,CNN_Glove,CNN_Glove_USE,CNN_w2V,CNN_w2V_USE,Individual_Sent,Average_Sent
reviews_Books,88.845,93.36,93.705,92.315,93.71,92.305,93.065,87.265,84.885
reviews_CDs_and_Vinyl,88.215,92.24,92.89,90.8,92.755,90.875,91.86,85.055,83.235
reviews_Cell_Phones_and_Accessories,90.155,96.365,96.375,94.585,95.94,95.12,95.8,90.735,88.46
reviews_Clothing_Shoes_and_Jewelry,91.61,96.265,96.455,95.47,96.315,95.57,95.94,91.345,89.405
reviews_Electronics,89.065,95.06,95.32,93.07,94.835,93.785,94.46,88.225,85.73
reviews_Health_and_Personal_Care,87.36,93.79,94.16,91.855,93.875,92.38,93.13,87.68,84.975
reviews_Home_and_Kitchen,90.045,95.7,95.765,94.38,95.765,95.065,95.365,89.44,86.8
reviews_Kindle_Store,90.695,94.655,94.96,93.645,94.795,93.695,94.215,90.425,88.58
reviews_Movies_and_TV,89.505,93.285,93.705,92.01,93.6,92.28,93.185,88.025,86.35
reviews_Sports_and_Outdoors,89.83,95.27,95.5,94.075,95.595,94.705,95.16,89.545,87.065
